In [0]:
#from google.colab import files
#src = list(files.upload().values())[0]
#open('classify_svhn.py','wb').write(src)

In [0]:
import argparse
import os
import torchvision
import torchvision.transforms as transforms
import torch
import classify_svhn
from classify_svhn import Classifier
import numpy as np

SVHN_PATH = "svhn"
PROCESS_BATCH_SIZE = 32


def get_sample_loader(path, batch_size):
    """
    Loads data from `[path]/samples`

    - Ensure that path contains only one directory
      (This is due ot how the ImageFolder dataset loader
       works)
    - Ensure that ALL of your images are 32 x 32.
      The transform in this function will rescale it to
      32 x 32 if this is not the case.

    Returns an iterator over the tensors of the images
    of dimension (batch_size, 3, 32, 32)
    """
    data = torchvision.datasets.ImageFolder(
        path,
        transform=transforms.Compose([
            transforms.Resize((32, 32), interpolation=2),
            classify_svhn.image_transform
        ])
    )
    data_loader = torch.utils.data.DataLoader(
        data,
        batch_size=batch_size,
        num_workers=2,
    )
    return data_loader


def get_test_loader(batch_size):
    """
    Downloads (if it doesn't already exist) SVHN test into
    [pwd]/svhn.

    Returns an iterator over the tensors of the images
    of dimension (batch_size, 3, 32, 32)
    """
    testset = torchvision.datasets.SVHN(
        SVHN_PATH, split='test',
        download=True,
        transform=classify_svhn.image_transform
    )
    testloader = torch.utils.data.DataLoader(
        testset,
        batch_size=batch_size,
    )
    return testloader


def extract_features(classifier, data_loader):
    """
    Iterator of features for each image.
    """
    with torch.no_grad():
        for x, _ in data_loader:
            h = classifier.extract_features(x).numpy()
            for i in range(h.shape[0]):
                yield h[i]

In [0]:
def calculate_fid_score(sample_feature_iterator,
                        testset_feature_iterator):
    """
    To be implemented by you!
    """
    testset = list(testset_feature_iterator)
    sample = list(sample_feature_iterator)
    
    mu_q = np.mean(sample,axis=0)
    mu_p = np.mean(testset,axis=0)
    sigma_q = np.cov(sample,rowvar=False)
    sigma_p = np.cov(testset,rowvar=False)
    
    fid = np.linalg.norm(mu_p-mu_q,ord=2)**2 + np.trace(sigma_p + sigma_q - 2*(sigma_q*sigma_p)**0.5)
    return fid


In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [0]:
sample_directory = '/content/drive/My Drive/ift6135_A3/VAE_samples/'
trained_model = '/content/drive/My Drive/ift6135_A3/svhn_classifier.pt'

In [0]:
classifier = torch.load(trained_model, map_location='cpu')
classifier.eval()
sample_loader = get_sample_loader(sample_directory, PROCESS_BATCH_SIZE)
sample_f = extract_features(classifier, sample_loader)

test_loader = get_test_loader(PROCESS_BATCH_SIZE)
test_f = extract_features(classifier, test_loader)

Using downloaded and verified file: svhn/test_32x32.mat


In [0]:
fid = calculate_fid_score(sample_f, test_f)
print("FID score:", fid)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in sqrt
  


FID score: 42711.26775143316
